In [1]:
import json
import logging

from tqdm import tqdm

from Questionnaire.PsycoTest import UNIVERSAL_TRAITS
from AgentFactory import AgentFactory
from utils import clear_json

logging.basicConfig(level=logging.WARNING)

agent_factory = AgentFactory()

/lab/solexa_sun/lab_members/yancui/miniconda3/envs/fm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd 

multi_list = pd.read_excel('TedoneItemAssignmentTable30APR21.xlsx')

In [3]:
multi_list['instrument'].unique()

array(['16PF', '6FPQ', '7FACTOR', 'AB5C', 'Barchard2001', 'BFAS',
       'BFAS-20', 'BIDR', 'BIS_BAS', 'Buss1980', 'Cacioppo1982', 'CAT-PD',
       'Chapman1986', 'CPI', 'Foa1998', 'Foa2002', 'Goldberg1999',
       'HEXACO_PI', 'Hoyle2002', 'HPI', 'HPI-HIC', 'IPIP-IPC', 'JPI',
       'Levenson1981', 'MPQ', 'NEO', 'NEO5-20', 'ORAIS', 'ORVIS',
       'Radloff1977', 'Rosenberg1965', 'Scheier1994', 'Snyder1974',
       'Span2002', 'TCI', 'VIA'], dtype=object)

In [5]:
multi_list['instrument'].value_counts()

instrument
AB5C             456
CPI              313
VIA              252
6FPQ             229
TCI              228
HEXACO_PI        223
CAT-PD           210
ORAIS            200
HPI-HIC          198
NEO              185
16PF             172
BFAS             125
JPI              109
MPQ               95
ORVIS             92
HPI               85
Barchard2001      68
BFAS-20           64
7FACTOR           62
NEO5-20           56
Levenson1981      55
BIS_BAS           42
BIDR              40
IPIP-IPC          32
Goldberg1999      31
Chapman1986       24
Hoyle2002         24
Radloff1977       22
Buss1980          21
Foa2002           21
Foa1998           20
Span2002          12
Cacioppo1982      10
Rosenberg1965     10
Snyder1974        10
Scheier1994        9
Name: count, dtype: int64

In [4]:
# config_path: str = "configs/conf_LLaVA.json"
# config_path: str = "configs/conf_LLaMA3.json"
# config_path: str = "configs/conf_DeepSeek.json"
config_path: str = "configs/conf_GPT.json"
config = json.load(open(config_path, "r"))
config = config["model"]

agent = agent_factory.create_agent(config)

In [8]:
from Questionnaire.IPIPtest import IPIPTest

qn = IPIPTest(multi_list, 'IPIP-IPC')

ProgressBar = tqdm(total=len(qn.questions), desc="Processing Questions")
for no, q in tqdm(enumerate(qn.questions)):
    ProgressBar.update(1)

    agent.reset_msg_history()
    agent.instruction = config["system_msg"]
    prompt = [
        {"type": "text", "text": q},
    ]
    response, text = agent.get_response(prompt)

    try:
        dict = json.loads(clear_json(text))
        qn.record_answer(no, dict)
    except Exception as e:
        print(f"Error: {e}")
        print(f"text: {text}")
        continue

final_output = qn.analyze()
qn.save_to_jsonl(f"output/BFI_{config['template_type']}.jsonl")

Processing Questions: 100%|██████████| 10/10 [01:29<00:00,  8.92s/it]
32it [01:22,  2.57s/it]


In [9]:
final_output 

{'Introversion': 3.75,
 'Warmth': 4.75,
 'Callousness': 2.5,
 'Disparagement': 2.0,
 'Dominance': 3.0,
 'Submissiveness': 4.0,
 'Gregariousness': 4.25,
 'Tolerance': 4.25}

In [7]:
from Questionnaire.HEXACO import HEXACO

hexaco = HEXACO()

ProgressBar = tqdm(total=len(hexaco.get_questions()), desc="Processing HEXACO Questions")
for no, q in tqdm(enumerate(hexaco.get_questions())):
    ProgressBar.update(1)

    agent.reset_msg_history()
    agent.instruction = config["system_msg"]
    prompt = [
        {"type": "text", "text": q},
    ]
    response, text = agent.get_response(prompt)

    try:
        dict = json.loads(clear_json(text))
        hexaco.record_answer(no, dict)
    except Exception as e:
        print(f"Error: {e}")
        print(f"text: {text}")
        continue

final_output_hex = hexaco.analyze()
hexaco.save_to_jsonl(f"output/HEXACO_{config['template_type']}.jsonl")

Processing Questions: 100%|██████████| 44/44 [02:42<00:00,  3.69s/it]
61it [01:43,  1.70s/it]
